In [71]:
#line intersection
import tkinter as tk
from tkinter import Canvas
from shapely.geometry import LineString

class IntersectionVisualizer:
    def __init__(self, master):
        self.master = master
        master.title("Line Intersection")
        master.configure(bg="pink") 

        self.canvas = Canvas(master, width=600, height=600, bg="black")
        self.canvas.pack()

        self.points = []

        self.canvas.bind("<Button-1>", self.on_click)

        self.btn_check_intersection_cramer = tk.Button(master, text="Check Intersection (Cramer's Rule)", command=self.calculate_intersection_cramer, state=tk.DISABLED, bg="blue", fg="white", font=("Arial", 12, "bold"))
        self.btn_check_intersection_cramer.pack(pady=5)

        self.btn_check_intersection_shapely = tk.Button(master, text="Check Intersection (Shapely)", command=self.calculate_intersection_shapely, state=tk.DISABLED, bg="green", fg="white", font=("Arial", 12, "bold"))
        self.btn_check_intersection_shapely.pack(pady=5)

        self.btn_check_intersection_ccw = tk.Button(master, text="Check Intersection (CCW)", command=self.calculate_intersection_ccw, state=tk.DISABLED, bg="orange", fg="white", font=("Arial", 12, "bold"))
        self.btn_check_intersection_ccw.pack(pady=5)

        self.btn_reset = tk.Button(master, text="Reset", command=self.reset, state=tk.DISABLED, bg="red", fg="white", font=("Arial", 12, "bold"))
        self.btn_reset.pack(pady=5)

        self.result_label = tk.Label(master, text="Intersection: ", bg="white", font=("Arial", 12, "bold"))
        self.result_label.pack()

    def on_click(self, event):
        x, y = event.x, event.y
        self.points.append((x, y))

        self.canvas.create_oval(x - 5, y - 5, x + 5, y + 5, fill="red")

        if len(self.points) == 4:
            self.btn_check_intersection_cramer.config(state=tk.NORMAL)
            self.btn_check_intersection_shapely.config(state=tk.NORMAL)
            self.btn_check_intersection_ccw.config(state=tk.NORMAL)
            self.btn_reset.config(state=tk.NORMAL)

    def reset(self):
        self.points = []
        self.canvas.delete("all")
        self.result_label.config(text="Intersection: ")

    def calculate_intersection_cramer(self):
        result, intersection_point = self.are_lines_intersecting_cramer(self.points[0], self.points[1], self.points[2], self.points[3])
        self.display_intersection_result(result, intersection_point)

    def calculate_intersection_shapely(self):
        if len(self.points) >= 4:
            result, intersection_point = self.are_lines_intersecting_shapely(self.points[0], self.points[1], self.points[2], self.points[3])
            self.display_intersection_result(result, intersection_point)

    def calculate_intersection_ccw(self):
        if len(self.points) >= 4:
            result, intersection_point = self.are_lines_intersecting_ccw(self.points[0], self.points[1], self.points[2], self.points[3])
            self.display_intersection_result(result, intersection_point)

    def display_intersection_result(self, result, intersection_point):
        result_text = f"Intersection Line Parameters:\n"
        result_text += f"Point 1: ({self.points[0][0]}, {self.points[0][1]})\n"
        result_text += f"Point 2: ({self.points[1][0]}, {self.points[1][1]})\n"
        result_text += f"Point 3: ({self.points[2][0]}, {self.points[2][1]})\n"
        result_text += f"Point 4: ({self.points[3][0]}, {self.points[3][1]})\n"

        if result:
            result_text += f"Intersection Point: {intersection_point}\n"
            self.canvas.create_oval(intersection_point[0] - 5, intersection_point[1] - 5,
                                    intersection_point[0] + 5, intersection_point[1] + 5, fill="green")
        else:
            result_text += "No Intersection\n"

        self.result_label.config(text=result_text)

        self.canvas.create_line(self.points[0][0], self.points[0][1], self.points[1][0], self.points[1][1], fill="blue")
        self.canvas.create_line(self.points[2][0], self.points[2][1], self.points[3][0], self.points[3][1], fill="blue")

    def are_lines_intersecting_cramer(self, p1, q1, p2, q2):
        A1 = q1[1] - p1[1]
        B1 = p1[0] - q1[0]
        C1 = A1 * p1[0] + B1 * p1[1]

        A2 = q2[1] - p2[1]
        B2 = p2[0] - q2[0]
        C2 = A2 * p2[0] + B2 * p2[1]

        determinant = A1 * B2 - A2 * B1

        if determinant == 0:
            return False, None

        x = (B2 * C1 - B1 * C2) / determinant
        y = (A1 * C2 - A2 * C1) / determinant

        if min(p1[0], q1[0]) <= x <= max(p1[0], q1[0]) and min(p1[1], q1[1]) <= y <= max(p1[1], q1[1]) \
                and min(p2[0], q2[0]) <= x <= max(p2[0], q2[0]) and min(p2[1], q2[1]) <= y <= max(p2[1], q2[1]):
            return True, (x, y)
        else:
            return False, None

    def are_lines_intersecting_shapely(self, p1, q1, p2, q2):
        from shapely.geometry import LineString

        line1 = LineString([p1, q1])
        line2 = LineString([p2, q2])

        intersection_exists = line1.intersects(line2)

        if intersection_exists:
            intersection_point = line1.intersection(line2)
            if intersection_point.is_empty:
                return False, None
            else:
                return True, (intersection_point.x, intersection_point.y)
        else:
            return False, None

    def are_lines_intersecting_ccw(self, p1, q1, p2, q2):
        def orientation(p, q, r):
            val = (q[1] - p[1]) * (r[0] - q[0]) - (q[0] - p[0]) * (r[1] - q[1])
            if val == 0:
                return 0
            return 1 if val > 0 else 2

        o1 = orientation(p1, q1, p2)
        o2 = orientation(p1, q1, q2)
        o3 = orientation(p2, q2, p1)
        o4 = orientation(p2, q2, q1)
        if o1 != o2 and o3 != o4:
            result, intersection_point = self.calculate_intersection_point_ccw(p1, q1, p2, q2)
            if min(p1[0], q1[0]) <= intersection_point[0] <= max(p1[0], q1[0]) and \
                    min(p1[1], q1[1]) <= intersection_point[1] <= max(p1[1], q1[1]) and \
                    min(p2[0], q2[0]) <= intersection_point[0] <= max(p2[0], q2[0]) and \
                    min(p2[1], q2[1]) <= intersection_point[1] <= max(p2[1], q2[1]):
                return True, intersection_point
            else:
                return False, None

        return False, None

    def calculate_intersection_point_ccw(self, p1, q1, p2, q2):
        A1 = q1[1] - p1[1]
        B1 = p1[0] - q1[0]
        C1 = A1 * p1[0] + B1 * p1[1]

        A2 = q2[1] - p2[1]
        B2 = p2[0] - q2[0]
        C2 = A2 * p2[0] + B2 * p2[1]

        determinant = A1 * B2 - A2 * B1

        if determinant == 0:
            return False, None

        x = (B2 * C1 - B1 * C2) / determinant
        y = (A1 * C2 - A2 * C1) / determinant

        return True, (x, y)


root = tk.Tk()
app = IntersectionVisualizer(root)
root.mainloop()

In [54]:
#first code
import tkinter as tk
import math
import timeit

def add_point(event):
    x, y = event.x, event.y
    points.append((x, y))
    canvas.create_oval(x - 3, y - 3, x + 3, y + 3, fill="white")

def orientation(p, q, r):
    val = (q[1] - p[1]) * (r[0] - q[0]) - (q[0] - p[0]) * (r[1] - q[1])
    if val == 0:
        return 0
    return 1 if val > 0 else 2
def add_point(event):
    x, y = event.x, event.y
    points.append((x, y))
    canvas.create_oval(x - 3, y - 3, x + 3, y + 3, fill="black")

def orientation(p, q, r):
    val = (q[1] - p[1]) * (r[0] - q[0]) - (q[0] - p[0]) * (r[1] - q[1])
    if val == 0:
        return 0
    return 1 if val > 0 else 2

def brute_force(points):
    n = len(points)
    if n < 3:
        return points

    hull = []

    start_point = min(points, key=lambda p: (p[1], p[0]))
    hull.append(start_point)

    sorted_points = sorted(points, key=lambda p: math.atan2(p[1] - start_point[1], p[0] - start_point[0]))

    for p in sorted_points:
        while len(hull) > 1 and orientation(hull[-2], hull[-1], p) != 2:
            hull.pop()
            brute_step_by_step(hull)

        hull.append(p)
        brute_step_by_step(hull)

    return hull

def jarvis_march(points):
    n = len(points)
    if n < 3:
        return points

    hull = []

    leftmost = min(points, key=lambda p: p[0])
    hull.append(leftmost)

    while True:
        endpoint = points[0]
        for i in range(1, n):
            if endpoint == leftmost or orientation(leftmost, endpoint, points[i]) == 2:
                endpoint = points[i]

        if endpoint == hull[0]:
            break
        else:
            hull.append(endpoint)

        leftmost = endpoint

    return hull

def quick_hull(points):
    def find_hull(p1, p2, points):
        sub_hull = [point for point in points if is_right(p1, p2, point)]
        if not sub_hull:
            return [p1, p2]

        furthest_point = max(sub_hull, key=lambda point: distance(p1, p2, point))
        return find_hull(p1, furthest_point, sub_hull) + find_hull(furthest_point, p2, sub_hull)

    if len(points) < 3:
        return points

    leftmost = min(points, key=lambda p: p[0])
    rightmost = max(points, key=lambda p: p[0])

    upper_hull = find_hull(leftmost, rightmost, points)
    lower_hull = find_hull(rightmost, leftmost, points)

    return upper_hull + lower_hull[1:-1]

def graham_scan(points):
    def orientation(p, q, r):
        val = (q[1] - p[1]) * (r[0] - q[0]) - (q[0] - p[0]) * (r[1] - q[1])
        if val == 0:
            return 0
        return 1 if val > 0 else 2

    n = len(points)
    if n < 3:
        return points

    hull = []

    pivot = min(points, key=lambda p: (p[1], p[0]))

    points.sort(key=lambda p: (math.atan2(p[1] - pivot[1], p[0] - pivot[0]), p))

    hull.append(points[0])
    hull.append(points[1])

    for i in range(2, n):
        while len(hull) > 1 and orientation(hull[-2], hull[-1], points[i]) != 2:
            hull.pop()
            draw_convex_hull(hull)
        hull.append(points[i])
        draw_convex_hull(hull)

    return hull

def jarvis_march_steps(points):
    def orientation(p, q, r):
        val = (q[1] - p[1]) * (r[0] - q[0]) - (q[0] - p[0]) * (r[1] - q[1])
        if val == 0:
            return 0
        return 1 if val > 0 else 2

    n = len(points)
    if n < 3:
        return [points]

    hull_steps = []
    hull = []

    leftmost = min(points, key=lambda p: p[0])
    hull.append(leftmost)
    hull_steps.append(list(hull))

    while True:
        endpoint = points[0]
        for i in range(1, n):
            if endpoint == leftmost or orientation(leftmost, endpoint, points[i]) == 2:
                endpoint = points[i]

        if endpoint == hull[0]:
            break
        else:
            hull.append(endpoint)
            hull_steps.append(list(hull))

        leftmost = endpoint

    return hull_steps


def graham_scan_steps(points):
    def orientation(p, q, r):
        val = (q[1] - p[1]) * (r[0] - q[0]) - (q[0] - p[0]) * (r[1] - q[1])
        if val == 0:
            return 0
        return 1 if val > 0 else 2

    n = len(points)
    if n < 3:
        return [points]

    hull_steps = []
    hull = []

    pivot = min(points, key=lambda p: (p[1], p[0]))

    points.sort(key=lambda p: (math.atan2(p[1] - pivot[1], p[0] - pivot[0]), p))

    hull.append(points[0])
    hull.append(points[1])
    hull_steps.append(list(hull))

    for i in range(2, n):
        while len(hull) > 1 and orientation(hull[-2], hull[-1], points[i]) != 2:
            hull.pop()
            hull_steps.append(list(hull))
        hull.append(points[i])
        hull_steps.append(list(hull))

    return hull_steps

def quick_hull_steps(points):
    def find_hull(p1, p2, points, convex_hull_steps):
        sub_hull = [point for point in points if is_right(p1, p2, point)]
        if not sub_hull:
            return []

        furthest_point = max(sub_hull, key=lambda point: distance(p1, p2, point))
        convex_hull_steps.append(convex_hull + [furthest_point] + find_hull(p1, furthest_point, sub_hull, convex_hull_steps))
        convex_hull_steps.append(convex_hull + find_hull(furthest_point, p2, sub_hull, convex_hull_steps))
        return [furthest_point]

    if len(points) < 3:
        return [points]

    leftmost = min(points, key=lambda p: p[0])
    rightmost = max(points, key=lambda p: p[0])

    convex_hull_steps = [points]

    find_hull(leftmost, rightmost, points, convex_hull_steps)

    return convex_hull_steps

def brute_step_by_step(hull):
    canvas.delete("convex_hull")
    for i in range(len(hull) - 1):
        x1, y1 = hull[i]
        x2, y2 = hull[i + 1]
        canvas.create_line(x1, y1, x2, y2, fill="red", width=2, tags=("convex_hull",))
        canvas.update()
        canvas.after(500) 

    if hull:
        x1, y1 = hull[-1]
        x2, y2 = hull[0]
        canvas.create_line(x1, y1, x2, y2, fill="red", width=2, tags=("convex_hull",))
        canvas.update()
        canvas.after(500)  

def run_algorithm_step_by_step():
    global convex_hull_steps, step_index
    algorithm = algorithm_var.get()
    if algorithm == "QuickHull":
        convex_hull_steps = quick_hull_steps(points)
    elif algorithm == "GrahamScan":
        convex_hull_steps = graham_scan_steps(points)
    elif algorithm == "JarvisMarch":
        convex_hull_steps = jarvis_march_steps(points)
    elif algorithm == "BruteForce":
        convex_hull_steps = brute_step_by_step(points)
        

    step_index = 0
    draw_step()
    
def draw_step():
    global step_index

    canvas.delete("all")
    for x, y in points:
        canvas.create_oval(x - 3, y - 3, x + 3, y + 3, fill="white")

    if step_index < len(convex_hull_steps):
        draw_convex_hull(convex_hull_steps[step_index])

    step_index += 1 
    if step_index <= len(convex_hull_steps):
        canvas.after_id = canvas.after(1000, draw_step)  
    
def run_algorithm():
    global convex_hull
    algorithm = algorithm_var.get()
    start_time = time.time()
    if algorithm == "QuickHull":
        convex_hull = quick_hull(points)
    elif algorithm == "GrahamScan":
        convex_hull = graham_scan(points)
    elif algorithm == "JarvisMarch":
        convex_hull = jarvis_march(points)
    elif algorithm == "BruteForce":
        convex_hull = brute_force(points)
    elapsed_time = time.time() - start_time
    canvas.delete("all")
    for x, y in points:
        canvas.create_oval(x - 3, y - 3, x + 3, y + 3, fill="white")

    draw_convex_hull(convex_hull)
    time_label.config(text=f"Time Complexity: {elapsed_time:.6f} seconds")
def draw_convex_hull(hull):
    canvas.delete("convex_hull")
    for i in range(len(hull) - 1):
        x1, y1 = hull[i]
        x2, y2 = hull[i + 1]
        canvas.create_line(x1, y1, x2, y2, fill="red", width=2, tags=("convex_hull",))

    if hull:
        x1, y1 = hull[-1]
        x2, y2 = hull[0]
        canvas.create_line(x1, y1, x2, y2, fill="red", width=2, tags=("convex_hull",))

def distance(p1, p2, p3):
    return abs((p2[0] - p1[0]) * (p1[1] - p3[1]) - (p1[0] - p3[0]) * (p2[1] - p1[1]))

def is_right(p1, p2, p3):
    return (p2[0] - p1[0]) * (p3[1] - p1[1]) - (p3[0] - p1[0]) * (p2[1] - p1[1]) < 0

def reset_canvas():
    global points, convex_hull, step_index
    points = []
    convex_hull = []
    step_index = 0
    canvas.delete("all")


if __name__ == "__main__":
    points = []
    convex_hull = []
    convex_hull_steps = []
    step_index = 0
    
    root = tk.Tk()
    root.title("Convex Hull Visualization")

    canvas = tk.Canvas(root, width=800, height=600, bg="grey")
    canvas.pack()

    canvas.bind("<Button-1>", add_point)

    algorithm_var = tk.StringVar(value="QuickHull")

    quick_hull_radio = tk.Radiobutton(root, text="QuickHull", variable=algorithm_var, value="QuickHull", fg="green", activebackground="blue")
    quick_hull_radio.pack(side=tk.LEFT)

    graham_scan_radio = tk.Radiobutton(root, text="GrahamScan", variable=algorithm_var, value="GrahamScan", fg="green", activebackground="blue")
    graham_scan_radio.pack(side=tk.LEFT)

    jarvis_march_radio = tk.Radiobutton(root, text="JarvisMarch", variable=algorithm_var, value="JarvisMarch", fg="green", activebackground="blue")
    jarvis_march_radio.pack(side=tk.LEFT)

    brute_force_radio = tk.Radiobutton(root, text="BruteForce", variable=algorithm_var, value="BruteForce", fg="green", activebackground="blue")
    brute_force_radio.pack(side=tk.LEFT)
    
    btn_run_algorithm = tk.Button(root, text="Run Algorithm", command=run_algorithm, fg="green", bg="pink")
    btn_run_algorithm.pack(side=tk.LEFT)
    btn_run_algorithm_step_by_step = tk.Button(root, text="Run Algorithm Step by Step", command=run_algorithm_step_by_step, fg="green", bg="pink")
    btn_run_algorithm_step_by_step.pack(side=tk.LEFT)
    
    time_label = tk.Label(root, text="Time Complexity: 0.000000 seconds", font=('Arial', 9), foreground="black")
    time_label.pack(side=tk.BOTTOM, pady=10)
    btn_reset = tk.Button(root, text="Reset", command=reset_canvas, fg="green", bg="pink")
    btn_reset.pack(side=tk.LEFT)

    root.mainloop()

In [34]:
#second code
import tkinter as tk
import math

def add_point(event):
    x, y = event.x, event.y
    points.append((x, y))
    canvas.create_oval(x - 3, y - 3, x + 3, y + 3, fill="black")

def andrews_monotone_chain_steps(points):
    def orientation(p, q, r):
        val = (q[1] - p[1]) * (r[0] - q[0]) - (q[0] - p[0]) * (r[1] - q[1])
        if val == 0:
            return 0
        return 1 if val > 0 else 2

    n = len(points)
    if n < 3:
        return [points]

    hull_steps = []
    points = sorted(points)
    lower_hull = []
    upper_hull = []

    for point in points:
        while len(lower_hull) >= 2 and orientation(lower_hull[-2], lower_hull[-1], point) != 2:
            lower_hull.pop()
        lower_hull.append(point)
        hull_steps.append(list(lower_hull) + list(upper_hull))

    for point in reversed(points):
        while len(upper_hull) >= 2 and orientation(upper_hull[-2], upper_hull[-1], point) != 2:
            upper_hull.pop()
        upper_hull.append(point)
        hull_steps.append(list(lower_hull) + list(upper_hull))

    return hull_steps

def run_algorithm_step_by_step():
    global convex_hull_steps, step_index
    algorithm = algorithm_var.get()
    if algorithm == "AndrewsMonotoneChain":
        convex_hull_steps = andrews_monotone_chain_steps(points)

    step_index = 0  
    draw_step()

def draw_step():
    global step_index

    canvas.delete("all")
    for x, y in points:
        canvas.create_oval(x - 3, y - 3, x + 3, y + 3, fill="white")

    if step_index < len(convex_hull_steps):
        draw_convex_hull(convex_hull_steps[step_index])

    step_index += 1  
    if step_index <= len(convex_hull_steps):
        canvas.after_id = canvas.after(1000, draw_step)  

def draw_convex_hull(hull):
    canvas.delete("convex_hull")
    for i in range(len(hull) - 1):
        x1, y1 = hull[i]
        x2, y2 = hull[i + 1]
        canvas.create_line(x1, y1, x2, y2, fill="red", width=2, tags=("convex_hull",))

    if hull:
        x1, y1 = hull[-1]
        x2, y2 = hull[0]
        canvas.create_line(x1, y1, x2, y2, fill="red", width=2, tags=("convex_hull",))

def reset_canvas():
    global points, step_index
    points = []
    step_index = 0
    canvas.delete("all")

if __name__ == "__main__":
    points = []
    convex_hull_steps = []
    step_index = 0
    
    root = tk.Tk()
    root.title("Andrew's Monotone Chain Visualization")

    canvas = tk.Canvas(root, width=800, height=600, bg="grey")
    canvas.pack()

    canvas.bind("<Button-1>", add_point)

    algorithm_var = tk.StringVar(value="AndrewsMonotoneChain")

    andrews_monotone_chain_radio = tk.Radiobutton(root, text="AndrewsMonotoneChain", variable=algorithm_var, value="AndrewsMonotoneChain", fg="green", activebackground="blue")
    andrews_monotone_chain_radio.pack(side=tk.LEFT)
    
    btn_run_algorithm_step_by_step = tk.Button(root, text="Run Algorithm Step by Step", command=run_algorithm_step_by_step, fg="green", bg="pink")
    btn_run_algorithm_step_by_step.pack(side=tk.LEFT)
    
    btn_reset = tk.Button(root, text="Reset", command=reset_canvas, fg="green", bg="pink")
    btn_reset.pack(side=tk.LEFT)

    root.mainloop()


In [55]:
#third code
import tkinter as tk
import math
import time

def add_point(event):
    x, y = event.x, event.y
    points.append((x, y))
    canvas.create_oval(x - 3, y - 3, x + 3, y + 3, fill="black")

def orientation(p, q, r):
    val = (q[1] - p[1]) * (r[0] - q[0]) - (q[0] - p[0]) * (r[1] - q[1])
    if val == 0:
        return 0
    return 1 if val > 0 else 2

def brute_force_steps(points):
    steps = []
    n = len(points)
    
    for i in range(n):
        for j in range(i + 1, n):
            is_valid = True
            for k in range(n):
                if k != i and k != j:
                    orientation_val = orientation(points[i], points[j], points[k])
                    if orientation_val == 1:
                        is_valid = False
                        break
            
            if is_valid:
                steps.append([points[i], points[j]])

    return steps

def brute_step_by_step(hull):
    canvas.delete("convex_hull")
    for i in range(len(hull) - 1):
        x1, y1 = hull[i]
        x2, y2 = hull[i + 1]
        canvas.create_line(x1, y1, x2, y2, fill="red", width=2, tags=("convex_hull",))
        canvas.update()
        canvas.after(500) 

    if hull:
        x1, y1 = hull[-1]
        x2, y2 = hull[0]
        canvas.create_line(x1, y1, x2, y2, fill="red", width=2, tags=("convex_hull",))
        canvas.update()
        canvas.after(500)  

def brute_force(points):
    n = len(points)
    if n < 3:
        return points

    hull = []

    start_point = min(points, key=lambda p: (p[1], p[0]))
    hull.append(start_point)

    sorted_points = sorted(points, key=lambda p: math.atan2(p[1] - start_point[1], p[0] - start_point[0]))

    for p in sorted_points:
        while len(hull) > 1 and orientation(hull[-2], hull[-1], p) != 2:
            hull.pop()
            brute_step_by_step(hull)

        hull.append(p)
        brute_step_by_step(hull)

    return hull


def run_algorithm_step_by_step():
    global convex_hull_steps, step_index
    algorithm = algorithm_var.get()
    if algorithm == "QuickHull":
        convex_hull_steps = quick_hull_steps(points)
    elif algorithm == "GrahamScan":
        convex_hull_steps = graham_scan_steps(points)
    elif algorithm == "JarvisMarch":
        convex_hull_steps = jarvis_march_steps(points)
    elif algorithm == "BruteForce":
        convex_hull_steps = brute_force_steps(points)

    step_index = 0 
    draw_step()

def draw_step():
    global step_index

    canvas.delete("all")
    for x, y in points:
        canvas.create_oval(x - 3, y - 3, x + 3, y + 3, fill="white")

    if step_index < len(convex_hull_steps):
        draw_convex_hull(convex_hull_steps[step_index])

    step_index += 1  

    if step_index <= len(convex_hull_steps):
        canvas.after_id = canvas.after(1000, draw_step)  

if __name__ == "__main__":
    points = []
    convex_hull = []
    convex_hull_steps = []
    step_index = 0
    
    root = tk.Tk()
    root.title("Convex Hull Visualization")

    canvas = tk.Canvas(root, width=800, height=600, bg="grey")
    canvas.pack()

    canvas.bind("<Button-1>", add_point)

    algorithm_var = tk.StringVar(value="QuickHull")

    quick_hull_radio = tk.Radiobutton(root, text="QuickHull", variable=algorithm_var, value="QuickHull", fg="green", activebackground="blue")
    quick_hull_radio.pack(side=tk.LEFT)

    graham_scan_radio = tk.Radiobutton(root, text="GrahamScan", variable=algorithm_var, value="GrahamScan", fg="green", activebackground="blue")
    graham_scan_radio.pack(side=tk.LEFT)

    jarvis_march_radio = tk.Radiobutton(root, text="JarvisMarch", variable=algorithm_var, value="JarvisMarch", fg="green", activebackground="blue")
    jarvis_march_radio.pack(side=tk.LEFT)

    brute_force_radio = tk.Radiobutton(root, text="BruteForce", variable=algorithm_var, value="BruteForce", fg="green", activebackground="blue")
    brute_force_radio.pack(side=tk.LEFT)
    
    btn_run_algorithm = tk.Button(root, text="Run Algorithm", command=run_algorithm, fg="green", bg="pink")
    btn_run_algorithm.pack(side=tk.LEFT)
    btn_run_algorithm_step_by_step = tk.Button(root, text="Run Algorithm Step by Step", command=run_algorithm_step_by_step, fg="green", bg="pink")
    btn_run_algorithm_step_by_step.pack(side=tk.LEFT)
    
    time_label = tk.Label(root, text="Time Complexity: 0.000000 seconds", font=('Arial', 9), foreground="black")
    time_label.pack(side=tk.BOTTOM, pady=10)
    btn_reset = tk.Button(root, text="Reset", command=reset_canvas, fg="green", bg="pink")
    btn_reset.pack(side=tk.LEFT)

    root.mainloop()
    


In [60]:
import tkinter as tk
import math
import time

def add_point(event):
    x, y = event.x, event.y
    points.append((x, y))
    canvas.create_oval(x - 3, y - 3, x + 3, y + 3, fill="white")

def orientation(p, q, r):
    val = (q[1] - p[1]) * (r[0] - q[0]) - (q[0] - p[0]) * (r[1] - q[1])
    if val == 0:
        return 0
    return 1 if val > 0 else 2

def brute_force_steps(points):
    steps = []
    n = len(points)
    
    for i in range(n):
        for j in range(i + 1, n):
            is_valid = True
            for k in range(n):
                if k != i and k != j:
                    orientation_val = orientation(points[i], points[j], points[k])
                    if orientation_val == 1:
                        is_valid = False
                        break
            
            if is_valid:
                steps.append([points[i], points[j]])

    return steps

def brute_step_by_step(hull):
    canvas.delete("convex_hull")
    for i in range(len(hull) - 1):
        x1, y1 = hull[i]
        x2, y2 = hull[i + 1]
        canvas.create_line(x1, y1, x2, y2, fill="red", width=2, tags=("convex_hull",))
        canvas.update()
        canvas.after(500) 

    if hull:
        x1, y1 = hull[-1]
        x2, y2 = hull[0]
        canvas.create_line(x1, y1, x2, y2, fill="red", width=2, tags=("convex_hull",))
        canvas.update()
        canvas.after(500)

def brute_force(points):
    n = len(points)
    if n < 3:
        return points

    hull = []

    start_point = min(points, key=lambda p: (p[1], p[0]))
    hull.append(start_point)

    sorted_points = sorted(points, key=lambda p: math.atan2(p[1] - start_point[1], p[0] - start_point[0]))

    for p in sorted_points:
        while len(hull) > 1 and orientation(hull[-2], hull[-1], p) != 2:
            hull.pop()
            brute_step_by_step(hull)

        hull.append(p)
        brute_step_by_step(hull)

    return hull


def run_algorithm_step_by_step():
    global convex_hull_steps, step_index
    algorithm = algorithm_var.get()
    if algorithm == "QuickHull":
        convex_hull_steps = quick_hull_steps(points)
    elif algorithm == "GrahamScan":
        convex_hull_steps = graham_scan_steps(points)
    elif algorithm == "JarvisMarch":
        convex_hull_steps = jarvis_march_steps(points)
    elif algorithm == "BruteForce":
        convex_hull_steps = brute_force_steps(points)
    elif algorithm == "AndrewsMonotoneChain":
        convex_hull_steps = andrews_monotone_chain_steps(points)

    step_index = 0 
    draw_step()


def draw_step():
    global step_index

    canvas.delete("all")
    canvas.config(bg="black")
    for x, y in points:
        canvas.create_oval(x - 3, y - 3, x + 3, y + 3, fill="white")

    if step_index < len(convex_hull_steps):
        draw_convex_hull(convex_hull_steps[step_index])

    step_index += 1  

    if step_index <= len(convex_hull_steps):
        canvas.after_id = canvas.after(1000, draw_step) 

if __name__ == "__main__":
    points = []
    convex_hull = []
    convex_hull_steps = []
    step_index = 0
    
    root = tk.Tk()
    root.title("Convex Hull Visualization")

    canvas = tk.Canvas(root, width=800, height=600, bg="black") 
    canvas.pack()

    canvas.bind("<Button-1>", add_point)

    algorithm_var = tk.StringVar(value="QuickHull")

    quick_hull_radio = tk.Radiobutton(root, text="QuickHull", variable=algorithm_var, value="QuickHull", fg="green", activebackground="blue")
    quick_hull_radio.pack(side=tk.LEFT)

    graham_scan_radio = tk.Radiobutton(root, text="GrahamScan", variable=algorithm_var, value="GrahamScan", fg="green", activebackground="blue")
    graham_scan_radio.pack(side=tk.LEFT)

    jarvis_march_radio = tk.Radiobutton(root, text="JarvisMarch", variable=algorithm_var, value="JarvisMarch", fg="green", activebackground="blue")
    jarvis_march_radio.pack(side=tk.LEFT)

    brute_force_radio = tk.Radiobutton(root, text="BruteForce", variable=algorithm_var, value="BruteForce", fg="green", activebackground="blue")
    brute_force_radio.pack(side=tk.LEFT)

    andrews_monotone_chain_radio = tk.Radiobutton(root, text="AndrewsMonotoneChain", variable=algorithm_var, value="AndrewsMonotoneChain", fg="green", activebackground="blue")
    andrews_monotone_chain_radio.pack(side=tk.LEFT)
    
    btn_run_algorithm = tk.Button(root, text="Run Algorithm", command=run_algorithm, fg="green", bg="pink")
    btn_run_algorithm.pack(side=tk.LEFT)
    
    btn_run_algorithm_step_by_step = tk.Button(root, text="Run Algorithm Step by Step", command=run_algorithm_step_by_step, fg="green", bg="pink")
    btn_run_algorithm_step_by_step.pack(side=tk.LEFT)

    time_label = tk.Label(root, text="Time Complexity: 0.000000 seconds", font=('Arial', 9), foreground="green")
    time_label.pack(side=tk.BOTTOM, pady=10)
    
    btn_reset = tk.Button(root, text="Reset", command=reset_canvas, fg="green", bg="pink")
    btn_reset.pack(side=tk.BOTTOM)

    root.mainloop()
